In [1]:
from fim import apriori
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
df_train

,name,duration_ms,explicit,popularity,artists,album_name,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,features_duration_ms,time_signature,n_beats,n_bars,processing,genre
0,Long Road,241760,0,46,Funki Porcini,Hed Phone Sex,0.690,0.513,5,-12.529,...,0.536000,0.2810,0.6350,142.221,241760,4.0,567.0,141.0,1.279305,0
1,"Daniâl My Son, Where Did You Vanish?",444000,0,0,Siavash Amini,A Trail of Laughters,0.069,0.196,1,-24.801,...,0.934000,0.3600,0.0336,75.596,444000,4.0,617.0,155.0,2.367412,1
2,Ondskapens Galakse,333506,0,3,Kvelertak,Nattesferd,0.363,0.854,2,-7.445,...,0.800000,0.2620,0.4890,116.628,333507,4.0,652.0,162.0,3.700483,2
3,Can't Look Away,269800,0,23,The Wood Brothers,One Drop of Truth,0.523,0.585,5,-5.607,...,0.001270,0.1170,0.3850,140.880,269800,4.0,619.0,155.0,1.279305,3
4,Thunderground,307651,0,25,The Darkraver;DJ Vince,Happy Hardcore Top 100,0.643,0.687,7,-11.340,...,0.096900,0.2990,0.6370,143.956,307652,4.0,733.0,183.0,4.067086,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,Outpt,432026,0,0,Autechre,EP7,0.554,0.657,1,-14.232,...,0.615000,0.1320,0.0335,117.319,432027,4.0,837.0,212.0,2.367412,15
14996,Natural Environment Sounds: To Improve Sleep a...,182746,0,44,Natural White Noise Relaxation,Baby Sleep White Noise: Ultimate Album to Help...,0.103,0.860,1,-19.627,...,0.940000,0.1230,0.0016,92.323,182747,3.0,277.0,92.0,2.367412,8
14997,Psychological Attack,400091,0,43,Droplex,Psychological Attack,0.799,0.535,1,-9.103,...,0.857000,0.0713,0.0708,126.000,400091,4.0,837.0,209.0,2.367412,9
14998,Don't Hate The 808 (feat. Lafa Taylor),197250,0,37,Bassnectar;Lafa Taylor,Noise vs Beauty,0.511,0.970,5,-2.299,...,0.000014,0.4020,0.2700,160.044,197250,4.0,513.0,130.0,1.279305,19


# Feature preprocessing (light)

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

genres = df_train['genre'].unique()
# creo un dizionario per tenerli associati ad un numero
genres_dict = dict(zip(genres, range(len(genres))))
# Mappo i generi con i numeri
df_train['genre'] = df_train['genre'].map(genres_dict)
df_test['genre'] = df_test['genre'].map(genres_dict)

df_train['explicit'] = df_train['explicit'].astype(int)
df_test['explicit'] = df_test['explicit'].astype(int)

#Calculating only for the missing values in time_signature column by dividing n_beats/n_bars and rounding the result
df_train.loc[df_train['time_signature'].isna(), 'time_signature'] = round(df_train['n_beats'] / df_train['n_bars'])

df_test.loc[df_test['time_signature'].isna(), 'time_signature'] = round(df_test['n_beats'] / df_test['n_bars'])


'''
from sklearn.preprocessing import MinMaxScaler

numerical_cols_train = df_train.select_dtypes(include=[np.number]).columns
numerical_cols_test = df_test.select_dtypes(include=[np.number]).columns


scaler = MinMaxScaler()

scaler.fit(df_train[numerical_cols_train])

scaled_numerical_train = scaler.transform(df_train[numerical_cols_train])
scaled_numerical_test = scaler.transform(df_test[numerical_cols_test])

df_train[numerical_cols_train] = scaled_numerical_train
df_test[numerical_cols_test] = scaled_numerical_test
'''

#Dropping popularity confidence column
df_train.drop(columns=['popularity_confidence'], axis=1, inplace=True)
df_test.drop(columns=['popularity_confidence'], axis=1, inplace=True)

# Load your dataset
df_copy = df_train.copy()
df_copy_test = df_test.copy()

# Drop unnecessary columns
columns_to_drop = ['name', 'artists', 'album_name']
df_copy.drop(columns=columns_to_drop, axis=1, inplace=True)
df_copy_test.drop(columns=columns_to_drop, axis=1, inplace=True)


# Divide the dataset: one with known mode and one with missing mode
data_with_mode = df_copy.dropna(subset=['mode'])
data_missing_mode = df_copy[df_copy['mode'].isna()]
data_missing_mode_test = df_copy_test[df_copy_test['mode'].isna()]

# Features and target variable for the known 'mode'
X_known = data_with_mode.drop(columns=['mode'])
y_known = data_with_mode['mode']

# Features for the rows with missing 'mode'
X_missing = data_missing_mode.drop(columns=['mode'])
X_missing_test = data_missing_mode_test.drop(columns=['mode'])

# Train a classifier (e.g., Random Forest) on data with known 'mode'
clf = RandomForestClassifier(n_estimators=500, random_state=42, max_features=20)
clf.fit(X_known, y_known)

# Separate known and unknown 'mode' in test set
test_known_mode = df_copy_test.dropna(subset=['mode'])
test_unknown_mode = df_copy_test[df_copy_test['mode'].isna()]

# Features and actual 'mode' values for known 'mode' part of test set
X_test_known = test_known_mode.drop(columns=['mode'])
y_test_actual = test_known_mode['mode']

# Predict 'mode' for the known part of the test set
y_test_pred = clf.predict(X_test_known)

# Calculate accuracy
accuracy = accuracy_score(y_test_actual, y_test_pred)
print("Accuracy on test set:", accuracy)

# Predict 'mode' for rows with missing 'mode' in training and test set
y_missing_pred = clf.predict(X_missing)
y_missing_pred_test = clf.predict(X_missing_test)

# Fill in the missing 'mode' values with predictions in training and test sets
df_train.loc[df_train['mode'].isna(), 'mode'] = y_missing_pred
df_test.loc[df_test['mode'].isna(), 'mode'] = y_missing_pred_test


Accuracy on test set: 0.717999434868607


In [6]:
df_train.drop(columns=['name', 'artists', 'album_name', 'features_duration_ms', 'processing'], axis=1, inplace=True)

In [30]:
df_train

,explicit,popularity,key,mode,time_signature,genre,durationBin,n_beatsBin,n_barsBin,danceabilityBin,energyBin,loudnessBin,speechinessBin,acousticnessBin,instrumentalnessBin,livenessBin,valenceBin,tempoBin
0,not_explicit,46,F,major,4.0_time_signature_time_signature,0,"(227826.0, 288903.0]_duration_duration","(461.0, 625.0]_n_beats_n_beats","(117.0, 159.0]_n_bars_n_bars","(0.58, 0.695]_danceability","(0.418, 0.625]_energy","(-49.532, -11.899]_loudness","(0.0601, 0.105]_speechiness","(0.3, 0.664]_acousticness","(0.089, 0.826]_instrumentalness","(0.168, 0.321]_liveness","(0.511, 0.715]_valence","(129.99, 149.82]_tempo"
1,not_explicit,0,C#,minor,4.0_time_signature_time_signature,1,"(288903.0, 4120258.0]_duration_duration","(461.0, 625.0]_n_beats_n_beats","(117.0, 159.0]_n_bars_n_bars","(-0.001, 0.441]_danceability","(-0.001, 0.418]_energy","(-49.532, -11.899]_loudness","(0.0601, 0.105]_speechiness","(0.0686, 0.3]_acousticness","(0.826, 1.0]_instrumentalness","(0.321, 0.994]_liveness","(-0.001, 0.155]_valence","(-0.001, 94.81]_tempo"
2,not_explicit,3,D,major,4.0_time_signature_time_signature,2,"(288903.0, 4120258.0]_duration_duration","(625.0, 7348.0]_n_beats_n_beats","(159.0, 2170.0]_n_bars_n_bars","(-0.001, 0.441]_danceability","(0.788, 0.912]_energy","(-8.37, -6.404]_loudness","(0.0351, 0.0446]_speechiness","(-0.001, 0.00394]_acousticness","(0.089, 0.826]_instrumentalness","(0.168, 0.321]_liveness","(0.328, 0.511]_valence","(94.81, 117.048]_tempo"
3,not_explicit,23,F,major,4.0_time_signature_time_signature,3,"(227826.0, 288903.0]_duration_duration","(461.0, 625.0]_n_beats_n_beats","(117.0, 159.0]_n_bars_n_bars","(0.441, 0.58]_danceability","(0.418, 0.625]_energy","(-6.404, -4.646]_loudness","(0.0601, 0.105]_speechiness","(0.00394, 0.0686]_acousticness","(0.00011, 0.089]_instrumentalness","(0.113, 0.168]_liveness","(0.328, 0.511]_valence","(129.99, 149.82]_tempo"
4,not_explicit,25,G,major,4.0_time_signature_time_signature,4,"(288903.0, 4120258.0]_duration_duration","(625.0, 7348.0]_n_beats_n_beats","(159.0, 2170.0]_n_bars_n_bars","(0.58, 0.695]_danceability","(0.625, 0.788]_energy","(-11.899, -8.37]_loudness","(0.0446, 0.0601]_speechiness","(-0.001, 0.00394]_acousticness","(0.089, 0.826]_instrumentalness","(0.168, 0.321]_liveness","(0.511, 0.715]_valence","(129.99, 149.82]_tempo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,not_explicit,0,C#,minor,4.0_time_signature_time_signature,15,"(288903.0, 4120258.0]_duration_duration","(625.0, 7348.0]_n_beats_n_beats","(159.0, 2170.0]_n_bars_n_bars","(0.441, 0.58]_danceability","(0.625, 0.788]_energy","(-49.532, -11.899]_loudness","(0.0446, 0.0601]_speechiness","(0.00394, 0.0686]_acousticness","(0.089, 0.826]_instrumentalness","(0.113, 0.168]_liveness","(-0.001, 0.155]_valence","(117.048, 129.99]_tempo"
14996,not_explicit,44,C#,major,3.0_time_signature_time_signature,8,"(180000.0, 227826.0]_duration_duration","(-0.001, 327.0]_n_beats_n_beats","(83.0, 117.0]_n_bars_n_bars","(-0.001, 0.441]_danceability","(0.788, 0.912]_energy","(-49.532, -11.899]_loudness","(0.0446, 0.0601]_speechiness","(0.3, 0.664]_acousticness","(0.826, 1.0]_instrumentalness","(0.113, 0.168]_liveness","(-0.001, 0.155]_valence","(-0.001, 94.81]_tempo"
14997,not_explicit,43,C#,major,4.0_time_signature_time_signature,9,"(288903.0, 4120258.0]_duration_duration","(625.0, 7348.0]_n_beats_n_beats","(159.0, 2170.0]_n_bars_n_bars","(0.695, 0.98]_danceability","(0.418, 0.625]_energy","(-11.899, -8.37]_loudness","(0.0446, 0.0601]_speechiness","(-0.001, 0.00394]_acousticness","(0.826, 1.0]_instrumentalness","(-0.001, 0.0913]_liveness","(-0.001, 0.155]_valence","(117.048, 129.99]_tempo"
14998,not_explicit,37,F,major,4.0_time_signature_time_signature,19,"(180000.0, 227826.0]_duration_duration","(461.0, 625.0]_n_beats_n_beats","(117.0, 159.0]_n_bars_n_bars","(0.441, 0.58]_danceability","(0.912, 1.0]_energy","(-4.646, 3.156]_loudness","(0.0446, 0.0601]_speechiness","(-0.001, 0.00394]_acousticness","(-

# Feature pattern mining preprocessing (heavy)

In [27]:
df_train['durationBin'] = pd.qcut(df_train['duration_ms'], 4)
df_train['n_beatsBin'] = pd.qcut(df_train['n_beats'], 4)
df_train['n_barsBin'] = pd.qcut(df_train['n_bars'], 4)
df_train['danceabilityBin'] = pd.qcut(df_train['danceability'], 4)
df_train['energyBin'] = pd.qcut(df_train['energy'], 5)
df_train['loudnessBin'] = pd.qcut(df_train['loudness'], 5)
df_train['speechinessBin'] = pd.qcut(df_train['speechiness'], 5)
df_train['acousticnessBin'] = pd.qcut(df_train['acousticness'], 5)
df_train['instrumentalnessBin'] = pd.qcut(df_train['instrumentalness'], 5, duplicates='drop')
df_train['livenessBin'] = pd.qcut(df_train['liveness'], 5)
df_train['valenceBin'] = pd.qcut(df_train['valence'], 5)
df_train['tempoBin'] = pd.qcut(df_train['tempo'], 5)

df_train.drop(columns=['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'], axis=1, inplace=True)
#df_train.drop(columns=['duration_ms', 'n_beats', 'n_bars'], axis=1, inplace=True)

KeyError: 'danceability'

In [29]:
df_train['durationBin'] = df_train['durationBin'].astype(str) + '_duration'
df_train['n_beatsBin'] = df_train['n_beatsBin'].astype(str) + '_n_beats'
df_train['n_barsBin'] = df_train['n_barsBin'].astype(str) + '_n_bars'
df_train['time_signature'] = df_train['time_signature'].astype(str) + '_time_signature'
df_train['danceabilityBin'] = df_train['danceabilityBin'].astype(str) + '_danceability'
df_train['energyBin'] = df_train['energyBin'].astype(str) + '_energy'
df_train['loudnessBin'] = df_train['loudnessBin'].astype(str) + '_loudness'
df_train['speechinessBin'] = df_train['speechinessBin'].astype(str) + '_speechiness'
df_train['acousticnessBin'] = df_train['acousticnessBin'].astype(str) + '_acousticness'
df_train['instrumentalnessBin'] = df_train['instrumentalnessBin'].astype(str) + '_instrumentalness'
df_train['livenessBin'] = df_train['livenessBin'].astype(str) + '_liveness'
df_train['valenceBin'] = df_train['valenceBin'].astype(str) + '_valence'
df_train['tempoBin'] = df_train['tempoBin'].astype(str) + '_tempo'

In [17]:
explicit_dict = {0: 'not_explicit', 1: 'explicit'}
df_train['explicit'] = df_train['explicit'].map(explicit_dict)

mode_dict = {0: 'minor', 1: 'major'}
df_train['mode'] = df_train['mode'].map(mode_dict)

key_dict = {0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E', 5: 'F', 6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'}
df_train['key'] = df_train['key'].map(key_dict)

# Pattern Mining - Apriori

In [37]:
# Convert DataFrame rows to list of lists
transactions = df_train.values.tolist()

# Convert inner lists to string type
transactions = [[str(item) for item in transaction] for transaction in transactions]

## Frequent itemsets - that means >= min_sup

In [40]:
supp = 2
zmin = 2

itemsets = apriori(transactions, target="s", supp=supp, zmin=zmin, report='S')
pd.DataFrame(itemsets, columns=['frequent_itemsets', 'support'])

,frequent_itemsets,support
0,"(25, 4.0_time_signature_time_signature)",2.013333
1,"(25, not_explicit)",2.040000
2,"(24, 4.0_time_signature_time_signature)",2.160000
3,"(24, 4.0_time_signature_time_signature, not_ex...",2.046667
4,"(24, not_explicit)",2.253333
...,...,...
26014,"((-0.001, 0.00011]_instrumentalness, not_expli...",36.113333
26015,"(major, 4.0_time_signature_time_signature)",58.373333
26016,"(major, 4.0_time_signature_time_signature, not...",55.160000
26017,"(major, not_explicit)",62.746667


## Closed Itemsets 

An item set is closed if non of its immediate supersets has the same support as the item set.

In [41]:
supp = 20
zmin = 2

itemsets = apriori(transactions, target="c", supp=supp, zmin=zmin, report='S')
pd.DataFrame(itemsets, columns=['closed_itemsets', 'support'])

,closed_itemsets,support
0,"((117.0, 159.0]_n_bars_n_bars, (461.0, 625.0]_...",22.146667
1,"((117.0, 159.0]_n_bars_n_bars, (461.0, 625.0]_...",21.580000
2,"((117.0, 159.0]_n_bars_n_bars, (461.0, 625.0]_...",20.253333
3,"((117.0, 159.0]_n_bars_n_bars, (461.0, 625.0]_...",20.793333
4,"((117.0, 159.0]_n_bars_n_bars, 4.0_time_signat...",22.800000
...,...,...
65,"((-0.001, 0.00011]_instrumentalness, not_expli...",36.113333
66,"(major, 4.0_time_signature_time_signature)",58.373333
67,"(major, 4.0_time_signature_time_signature, not...",55.160000
68,"(major, not_explicit)",62.746667


## Maximal Itemsets

In [47]:
supp = 25
zmin = 2

itemsets = apriori(transactions, target="m", supp=supp, zmin=zmin, report='S')
pd.DataFrame(itemsets, columns=['maximal_itemsets', 'support'])

,maximal_itemsets,support
0,"(minor, 4.0_time_signature_time_signature, not...",27.480000
1,"((-0.001, 0.00011]_instrumentalness, major, 4....",26.653333
2,"((-0.001, 0.00011]_instrumentalness, major, no...",27.153333
3,"((-0.001, 0.00011]_instrumentalness, 4.0_time_...",33.180000
4,"(major, 4.0_time_signature_time_signature, not...",55.160000
